<a href="https://colab.research.google.com/github/vidhya2324/Machine_learning/blob/main/News_Article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Library

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
import numpy as np
import pandas as pd
import string

In [2]:
lemma = WordNetLemmatizer() # Inisialisasi WordNetLemmatizer
# Download data untuk pemrosesan bahasa alami
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
!kaggle datasets download -d timilsinabimal/newsarticlecategories

Dataset URL: https://www.kaggle.com/datasets/timilsinabimal/newsarticlecategories
License(s): CC0-1.0
  0% 0.00/9.94M [00:00<?, ?B/s]
100% 9.94M/9.94M [00:00<00:00, 108MB/s]


#Load dataset

In [4]:
!unzip \*.zip && rm *.zip


Archive:  newsarticlecategories.zip
replace news-article-categories.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
data = pd.read_csv('/content/news-article-categories.csv') # Membaca file CSV
df = pd.DataFrame(data) # Membuat DataFrame dari data
df.tail(10)

,category,title,body
6867,WOMEN,Your Compliments Are Gross And So Are You,Upon reading my fair share of social media pos...
6868,WOMEN,Ghoul Power!,"Picking out a Halloween costume can be tough, ..."
6869,WOMEN,IVF Confessions: in Defense of Egg Freezing,When the media reported earlier this month tha...
6870,WOMEN,e Need To Stop Judging Other Women for Their H...,"It's October, and the pumpkin spice lattes are..."
6871,WOMEN,Women Who Run,As I listened to Kathrine Switzer describe her...
6872,WOMEN,Casually Fearless: Why Millennials Are Natural...,I still think about that Tuesday night dinner ...
6873,WOMEN,Happy Birthday To Us,I remember the morning of my high school gradu...
6874,WOMEN,The Culture of Love,"My husband, Gene, doesn't wear pajamas. I aske..."
6875,WOMEN,"Carpe Diem, Oprah Style","\nBy AntonioGuillem, via ThinkStock\nBy Lisa ..."
6876,WOMEN,My Month of Online Dating Detox,My name is Jamie and I'm an online date-aholic...


#Removing NULL Values¶

In [6]:
df.isnull().sum()

category    0
title       0
body        5
dtype: int64

In [8]:
df = df.dropna()
df = df.dropna(axis=1)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6877 entries, 0 to 6876
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  6877 non-null   object
 1   title     6877 non-null   object
 2   body      6872 non-null   object
dtypes: object(3)
memory usage: 161.3+ KB


#Encoding Category

In [9]:
df['category'].value_counts()

category
ARTS & CULTURE    1001
BUSINESS           501
ENTERTAINMENT      501
ENVIRONMENT        501
POLITICS           501
RELIGION           501
SPORTS             501
TECH               501
WOMEN              501
EDUCATION          490
COMEDY             376
SCIENCE            350
MEDIA              347
CRIME              300
Name: count, dtype: int64

In [10]:
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(df['category'])
df['label'] = label_encoder.transform(df['category'])


#Text Preperation

In [11]:
df['body']=df['body'].str.lower()
df.head()

,category,title,body,label
0,ARTS & CULTURE,Modeling Agencies Enabled Sexual Predators For...,"in october 2017, carolyn kramer received a dis...",0
1,ARTS & CULTURE,Actor Jeff Hiller Talks “Bright Colors And Bol...,this week i talked with actor jeff hiller abou...,0
2,ARTS & CULTURE,New Yorker Cover Puts Trump 'In The Hole' Afte...,the new yorker is taking on president donald t...,0
3,ARTS & CULTURE,Man Surprises Girlfriend By Drawing Them In Di...,"kellen hickey, a 26-year-old who lives in huds...",0
4,ARTS & CULTURE,This Artist Gives Renaissance-Style Sculptures...,there’s something about combining the traditio...,0


#Remove HTML Tags


In [12]:
# Import modul BeautifulSoup
from bs4 import BeautifulSoup

# Mengecek apakah ada tag html didalam teks

# Mendefinisikan Fungsi 'has_html_tags'
def has_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return bool(soup.find())

df['has_html_tags'] = df['body'].apply(has_html_tags)
df.head()


<ipython-input-12-1f9fc603bbc6>:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


,category,title,body,label,has_html_tags
0,ARTS & CULTURE,Modeling Agencies Enabled Sexual Predators For...,"in october 2017, carolyn kramer received a dis...",0,False
1,ARTS & CULTURE,Actor Jeff Hiller Talks “Bright Colors And Bol...,this week i talked with actor jeff hiller abou...,0,False
2,ARTS & CULTURE,New Yorker Cover Puts Trump 'In The Hole' Afte...,the new yorker is taking on president donald t...,0,False
3,ARTS & CULTURE,Man Surprises Girlfriend By Drawing Them In Di...,"kellen hickey, a 26-year-old who lives in huds...",0,False
4,ARTS & CULTURE,This Artist Gives Renaissance-Style Sculptures...,there’s something about combining the traditio...,0,False


In [13]:
count_true = df['has_html_tags'].sum()
count_true


19

In [ ]:
df = df.drop(df[df['has_html_tags']].index)

In [14]:
df = df.drop('has_html_tags', axis=1)
df


,category,title,body,label
0,ARTS & CULTURE,Modeling Agencies Enabled Sexual Predators For...,"in october 2017, carolyn kramer received a dis...",0
1,ARTS & CULTURE,Actor Jeff Hiller Talks “Bright Colors And Bol...,this week i talked with actor jeff hiller abou...,0
2,ARTS & CULTURE,New Yorker Cover Puts Trump 'In The Hole' Afte...,the new yorker is taking on president donald t...,0
3,ARTS & CULTURE,Man Surprises Girlfriend By Drawing Them In Di...,"kellen hickey, a 26-year-old who lives in huds...",0
4,ARTS & CULTURE,This Artist Gives Renaissance-Style Sculptures...,there’s something about combining the traditio...,0
...,...,...,...,...
6872,WOMEN,Casually Fearless: Why Millennials Are Natural...,i still think about that tuesday night dinner ...,13
6873,WOMEN,Happy Birthday To Us,i remember the morning of my high school gradu...,13
6874,WOMEN,The Culture of Love,"my husband, gene, doesn't wear pajamas. i aske...",13
6875,WOMEN,"Carpe Diem, Oprah Style","\nby antonioguillem, via thinkstock\nby lisa ...",13


#Remove Emojies

In [17]:
import regex # Mengimpor modul regex yang digunakan untuk bekerja dengan ekspresi reguler yang mendukung Unicode

# Mendefinisikan Fungsi 'has_emoji'
def has_emoji(text):
    emoji_pattern = regex.compile(r'\p{Emoji}', flags=regex.UNICODE)
    return bool(emoji_pattern.search(text))


has_emojis =  df['body'].apply(has_emoji)

has_emojis


0       True
1       True
2       True
3       True
4       True
        ... 
6872    True
6873    True
6874    True
6875    True
6876    True
Name: body, Length: 6872, dtype: bool

In [18]:
has_emojis.sum()

6138

In [19]:
# Mendefinisikan fungsi remove_emojis
def remove_emojis(text):
    emoji_pattern = regex.compile(r'\p{Emoji}', flags=regex.UNICODE)
    return emoji_pattern.sub('', text)

df['body'] = df['body'].apply(remove_emojis)

In [20]:
has_emojis =  df['body'].apply(has_emoji)
has_emojis

0       False
1       False
2       False
3       False
4       False
        ...  
6872    False
6873    False
6874    False
6875    False
6876    False
Name: body, Length: 6872, dtype: bool

In [21]:
has_emojis.sum()

0

#Remove urls

In [22]:
import re
def remove_url(text):
    pattern=re.compile(r'https?://\S+|www\.S+')
    return pattern.sub(r'',text)
df["body"]=df["body"].apply(remove_url)


#Tokenisasi, Remove punctuation, Remove stopwords

In [23]:
from string import punctuation

string.punctuation
def DataPrep(text) :
    tokens = nltk.word_tokenize(text)

    punc = list(punctuation)
    words = [w for w in tokens if w not in punc]

    stop_words = set(stopwords.words('english'))


    words = [lemma.lemmatize(w) for w in words]

    text = ' '.join(words)

    return text


#TF-IDF Vectorizer

In [24]:
x = df['body'] = df['body'].apply(DataPrep)
y = df['label']
vectorizer = TfidfVectorizer(stop_words = 'english', max_features=6000)
vectorized_x = vectorizer.fit_transform(x.values.astype('U'))


#Split

In [25]:
x_train, x_test, y_train, y_test = train_test_split(vectorized_x, y, test_size=0.1, stratify=y, random_state=0)

In [26]:
# Impor library
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(x_train, y_train)

os = pd.DataFrame(list(zip([x[0] for x in X_resampled], y_resampled)), columns=['body', 'Label'])

os['Label'].value_counts()


Label
8     900
3     900
0     900
4     900
5     900
9     900
7     900
2     900
6     900
10    900
13    900
11    900
1     900
12    900
Name: count, dtype: int64

#Model

In [27]:
#SVM
from sklearn.svm import SVC

classifier = SVC(random_state=42)

classifier.fit(X_resampled, y_resampled)

y_pred = classifier.predict(x_test)

SVM_ACCURACY = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print(cm)

print(classification_report(y_test, y_pred))

Accuracy: 0.8197674418604651
[[88  0  1  0  0  1  2  0  1  1  0  0  2  5]
 [ 2 33  0  0  1  0  3  0  4  0  0  0  6  1]
 [ 2  0 26  0  1  5  3  0  1  0  0  0  0  0]
 [ 0  1  0 24  2  0  0  0  1  1  0  0  0  1]
 [ 4  2  0  0 40  0  0  0  2  0  1  0  0  0]
 [ 2  0  1  0  0 44  0  1  0  0  0  2  0  0]
 [ 0  0  0  0  0  0 48  0  0  0  1  0  0  1]
 [ 1  1  0  0  0  1  1 28  0  0  0  0  0  3]
 [ 0  0  0  1  1  2  0  1 43  0  0  0  1  1]
 [ 1  0  0  0  0  0  0  0  1 47  0  0  0  1]
 [ 3  0  0  0  1  0  7  0  0  0 22  1  0  1]
 [ 1  1  0  1  0  2  0  0  0  0  0 44  0  1]
 [ 3  4  0  0  0  0  1  1  1  0  0  1 38  1]
 [ 3  2  0  2  0  1  0  1  1  1  0  0  0 39]]
              precision    recall  f1-score   support

           0       0.80      0.87      0.83       101
           1       0.75      0.66      0.70        50
           2       0.93      0.68      0.79        38
           3       0.86      0.80      0.83        30
           4       0.87      0.82      0.84        49
           5   

#Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42, max_iter=200)

model.fit(X_resampled, y_resampled)

y_pred = model.predict(x_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.809593023255814
[[83  1  2  0  0  2  2  2  1  1  0  0  2  5]
 [ 0 31  0  1  1  0  5  0  5  0  0  0  6  1]
 [ 1  0 28  0  1  4  2  0  0  0  2  0  0  0]
 [ 0  0  0 26  2  0  0  0  0  1  0  0  0  1]
 [ 3  2  0  1 40  0  0  0  2  0  1  0  0  0]
 [ 1  0  0  0  0 44  0  1  0  1  0  2  0  1]
 [ 0  2  0  1  1  0 44  0  0  0  1  0  0  1]
 [ 0  1  0  0  0  0  1 29  0  1  0  0  1  2]
 [ 0  1  0  3  1  1  0  1 42  0  0  0  1  0]
 [ 1  0  0  0  0  0  0  0  1 47  0  0  0  1]
 [ 0  0  0  0  1  0  6  0  0  0 28  0  0  0]
 [ 0  2  1  1  0  1  0  0  0  0  0 43  0  2]
 [ 2  5  1  0  0  1  1  1  1  0  0  1 37  0]
 [ 1  2  0  3  1  2  0  1  1  3  1  0  0 35]]
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.82      0.86       101
           1       0.66      0.62      0.64        50
           2       0.88      0.74      0.80        38
           3       0.72      0.87      0.79        30
           4       0.83      0.82      0.82    

#Stochastic Gradient Descent


In [29]:
from sklearn.linear_model import SGDClassifier

classifier = SGDClassifier(random_state=0)

classifier.fit(X_resampled, y_resampled)
y_pred = classifier.predict(x_test)
# Evaluasi model
SGD_ACCURACY = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred) #  confusion matrix
print(cm) #  confusion matrix

print("Classification Report:") #  classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.8212209302325582
[[86  2  2  0  0  1  1  1  1  1  0  1  1  4]
 [ 1 32  0  1  1  0  4  0  4  0  0  0  6  1]
 [ 1  0 27  0  1  2  2  1  1  1  2  0  0  0]
 [ 0  0  0 27  2  0  0  0  0  1  0  0  0  0]
 [ 1  2  0  0 44  0  0  0  1  0  1  0  0  0]
 [ 1  0  1  0  0 44  0  1  0  1  0  2  0  0]
 [ 1  4  0  1  1  0 40  0  0  0  2  0  0  1]
 [ 0  1  0  0  0  0  1 28  1  1  0  0  0  3]
 [ 0  0  0  2  1  3  1  1 41  0  0  0  1  0]
 [ 1  0  0  0  0  0  0  0  0 49  0  0  0  0]
 [ 0  0  1  1  1  0  3  0  0  0 29  0  0  0]
 [ 0  2  1  1  0  0  0  0  0  0  0 44  0  2]
 [ 1  3  1  0  0  1  0  0  1  0  1  1 40  1]
 [ 1  3  0  2  2  1  0  1  2  2  1  1  0 34]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.85      0.88       101
           1       0.65      0.64      0.65        50
           2       0.82      0.71      0.76        38
           3       0.77      0.90      0.83        30
           4       0.83      0.90      0.86   

In [35]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the lemmatizer
lemma = WordNetLemmatizer()

# Define preprocessing functions
def DataPrep(text):
    tokens = nltk.word_tokenize(text)
    punc = list(punctuation)
    words = [w for w in tokens if w not in punc]
    stop_words = set(stopwords.words('english'))
    words = [lemma.lemmatize(w) for w in words if w not in stop_words]
    return ' '.join(words)

# Load the vectorizer (make sure it's the same as used for training)
vectorizer = TfidfVectorizer(stop_words='english', max_features=6000)

# Load the trained models (ensure they are the models you've trained)
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier

# Load your models (make sure these are loaded from the saved state or retrained)
svm_model = SVC(random_state=42)
log_reg_model = LogisticRegression(random_state=42, max_iter=200)
sgd_model = SGDClassifier(random_state=0)
stacking_model = StackingClassifier(
    estimators=[
        ('svc', SVC(random_state=42)),
        ('dt', DecisionTreeClassifier(random_state=0)),
        ('lr', LogisticRegression(random_state=0))
    ],
    final_estimator=SVC()
)

# Load or train your models if needed here
# For example:
# svm_model.fit(X_resampled, y_resampled)
# log_reg_model.fit(X_resampled, y_resampled)
# sgd_model.fit(X_resampled, y_resampled)
# stacking_model.fit(X_resampled, y_resampled)

def predict_category(text):
    preprocessed_text = DataPrep(text)
    vectorized_text = vectorizer.transform([preprocessed_text])

    svm_pred = predict_svm(user_input)
    log_reg_pred = predict_log_reg(user_input)
    sgd_pred = predict_sgd(user_input)
    stacking_pred = predict_stacking(user_input)
    return svm_pred, log_reg_pred, sgd_pred, stacking_pred


In [ ]:

user_input = input("Enter the news article text: ")
svm_pred, log_reg_pred, sgd_pred, stacking_pred = predict_category(user_input)

print(f"SVM Prediction: {svm_pred}")
print(f"Logistic Regression Prediction: {log_reg_pred}")
print(f"SGD Prediction: {sgd_pred}")
print(f"Stacking Classifier Prediction: {stacking_pred}")


#Ensemble Stacking Classifier With SVM, Decision Tree, and logistic regression

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# model
model1 = SVC(random_state=42)
model2 = DecisionTreeClassifier(random_state=0)
model3 = LogisticRegression(random_state=0)

# StackingClassifier
ensembleSC = StackingClassifier(estimators=[('svc', model1), ('dt', model2), ('lr', model3)], final_estimator=SVC())

#  model
ensembleSC.fit(X_resampled, y_resampled)

# Evaluate model
print(f"Akurasi training: {ensembleSC.score(x_train, y_train)}")

y_pred = ensembleSC.predict(x_test)
print(f"Akurasi prediksi: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))